In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import sys
import iprompt.data
from transformers import AutoTokenizer, OPTForCausalLM, AutoModelForCausalLM
from iprompt.data import TASKS_GALACTICA
import transformers
from imodelsx import explain_dataset_iprompt, get_add_two_numbers_dataset

/home/chansingh/iprompt/iprompt/data_utils/galactica.py:34: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  control = d.loc[(d.sum(axis=1) == 0) & (d.isna().sum(axis=1) == 0)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tox = d.loc[(d.sum(axis=1) == 1) & (d.iloc[:, tox_target] == 1)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tox'].iloc[n:] = 0

# BBBP

In [6]:
task = TASKS_GALACTICA['bbbp']
df = task['gen_func']()
input_strings = df['input'].values
output_strings = df['output'].values
input_strings[0]

/home/chansingh/iprompt/iprompt/data_utils/galactica.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name.loc[df.name.str.isnumeric()] = 'Compound-'+df.name.loc[df.name.str.isnumeric()] # rename compounds that are just numbers


'Here is a compound:\nPropanolol\nAnswer:'

In [ ]:
# checkpoint = 'facebook/galactica-6.7b'
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer.eos_token = tokenizer.eos_token or 0
# lm = AutoModelForCausalLM.from_pretrained(
#     checkpoint, output_hidden_states=False, pad_token_id=tokenizer.eos_token_id
# )
# lm = lm.to('cuda')

Using eos_token, but it is not set yet.


In [8]:
# get a simple dataset of adding two numbers

# explain the relationship between the inputs and outputs
# with a natural-language prompt string
prompts, metadata = explain_dataset_iprompt(
    input_strings=input_strings,
    output_strings=output_strings,
    # checkpoint='EleutherAI/gpt-j-6B', # which language model to use
    checkpoint="facebook/galactica-6.7b", # which language model to use
    preprefix='Answer P if the compound can ',
    num_learned_tokens=6, # how long of a prompt to learn
    n_shots=5, # shots per example
    n_epochs=15, # how many epochs to search
    verbose=0, # how much to print
    llm_float16=True, # whether to load the model in float_16
    seed=42,
    llm_candidate_regeneration_prompt_start='Data:',
    # llm_candidate_regeneration_prompt_end='What function of the compound is being asked about?', # default is "Prompt:"
    # lm=lm,
)

Using eos_token, but it is not set yet.


trying to convert facebook/galactica-6.7b to float16...


# Tox21

In [3]:
task = TASKS_GALACTICA['tox21_0']
df = task['gen_func']()
input_strings = df['input'].values
output_strings = df['output'].values
input_strings[0], output_strings[0]

/home/chansingh/iprompt/iprompt/data_utils/galactica.py:34: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  control = d.loc[(d.sum(axis=1) == 0) & (d.isna().sum(axis=1) == 0)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tox = d.loc[(d.sum(axis=1) == 1) & (d.iloc[:, tox_target] == 1)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tox'].iloc[n:] = 0

('Here is a compound:\nTOX3819\nAnswer:', ' No.\n\n')

In [4]:
# get a simple dataset of adding two numbers

# explain the relationship between the inputs and outputs
# with a natural-language prompt string
prompts, metadata = explain_dataset_iprompt(
    input_strings=input_strings,
    output_strings=output_strings,
    # checkpoint='EleutherAI/gpt-j-6B', # which language model to use
    checkpoint="facebook/galactica-6.7b", # which language model to use
    preprefix='Answer Yes if the compound is',
    num_learned_tokens=6, # how long of a prompt to learn
    n_shots=5, # shots per example
    n_epochs=15, # how many epochs to search
    verbose=0, # how much to print
    llm_float16=True, # whether to load the model in float_16
    seed=42,
    llm_candidate_regeneration_prompt_start='Data:',
    # llm_candidate_regeneration_prompt_end='What function of the compound is being asked about?', # default is "Prompt:"
    # lm=lm,
)

Using eos_token, but it is not set yet.


trying to convert facebook/galactica-6.7b to float16...
loading model...
Training with 2 possible answers / random acc 50.0% / majority acc 50.6%
Beginning epoch 0


  0%|          | 0/157 [00:00<?, ?it/s]

population_pool: [' be avoided; otherwise, answer', ' at least one of these targets', ' both, False otherwise (', ' human health (i)', ' either liver or kidney and an', ' produce human health concern (', ' your body; otherwise answer no', ' humans for any reason whatso', ' at least one of these targets', ' either liver or kidney and an', ' either liver or kidney and an', ' both, False otherwise (']


Loss = 3.738:   1%|          | 1/157 [00:09<24:13,  9.32s/it]

...

population_pool: [', or answer No otherwiseShow', ' be avoided; answer No otherwise', ' humans, otherwise answer No ', ' use and/or answer No', ' human beings and answerNo otherwise', ' answer no and vice versa,', ' animals, but answer no otherwise', ' cause harm and answer no otherwise', ' rats, while answer no for', ' your health, otherwise answer no', ' both, False otherwiseYesNo', ' either organism, else answer no']


Loss = 2.166:   8%|▊         | 13/157 [01:46<19:39,  8.19s/it]

population_pool: [', or answer No otherwiseShow', ' be avoided; answer No otherwise', ' humans, otherwise answer No ', ' use and/or answer No', ' human beings and answerNo otherwise', ' answer no and vice versa,', ' your body and answerNo for', ' animals, but answer no otherwise', ' cause harm and answer no otherwise', ' both, False otherwiseYesNo', ' rats, while answer no for', ' test it, or answer no']


Loss = 2.166:   8%|▊         | 13/157 [01:54<21:05,  8.79s/it]


KeyboardInterrupt: 

# Demo generations

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-6.7b")
model = OPTForCausalLM.from_pretrained("facebook/galactica-6.7b", device_map="auto", torch_dtype=torch.float16)

In [3]:
def gen(model, input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=200)
    return tokenizer.decode(outputs[0])

In [4]:
gen(model, "[START_I_SMILES]")

'[START_I_SMILES]CC1=CC=C(C=C1)C2=NN=C(N2C3=CC=CC=C3)SCC(=O)NC4=CC=CC=C4OC[END_I_SMILES]\n\n### Molecular Formula\n\nC24H22N4O2S\n\n## Chemical and Physical Properties\n\nThe following are chemical properties for 2-[[4,5-bis(p-tolyl)-1,2,4-triazol-3-yl]sulfanyl]-N-(2-methoxyphenyl)acetamide.\n\n### Computed Properties\n\n| Property Name | Property Value\n| --- | ----------- |\n| Molecular Weight | 430.5\n| XLogP3-AA Log P | 4.9\n| Hydrogen Bond Donor Count | 1\n| Hydrogen Bond Acceptor Count |'